In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species, GBsource
from utils import *
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.45,                 # Elongation factor
    delta       = 0.35,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04                  # position of the LCFS (x_inner)
)

simulation.set_data_param(
    expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/',
    g0simdir   = home_dir+'/g0_sim/',
    simname    = '48x32x16x16x6_nunifvg',
    simdir     = 'gk_tcv_posD_iwl_3x2v_AH/',
    fileprefix = 'gk_tcv_posD_iwl_3x2v_D02',
    wkdir      = 'wk/'
)

# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)
# Check available frames
tfs = find_available_frames(simulation,'field')
print(tfs)
#--------- Set up some normalizations for the plots
#-- Time scale
simulation.set_normalization(key='t',scale=1e-6,shift=0,symbol='t',units=r'$\mu s$')
#-- Radial length
# Lref   = simulation.species['ion'].rho; xshift = 0; xsymbol = r'$x/\rho_s$'; xunits = ''
Lref   = 1.0; xshift = simulation.geom_param.x_LCFS; xsymbol = r'$R-R_{LCFS}$'; xunits = 'm'
simulation.set_normalization(key='x',scale=Lref,shift=xshift,symbol=xsymbol,units=xunits)
#-- Poloidal length
Lref   = simulation.species['ion'].rho
yshift = 0
ysymbol= r'$y/\rho_s$'
yunits = ''
simulation.set_normalization(key='y',scale=Lref,shift=0,symbol=ysymbol,units=yunits)
#-- Temperature normalization
Tref_i = simulation.phys_param.eV/simulation.species['ion'].m
Tref_e = simulation.phys_param.eV/simulation.species['elc'].m
simulation.set_normalization(key='upare', scale=elc.vt,shift=0,symbol=r'$u_{\parallel e}/v_{te}$',units='')
simulation.set_normalization(key='upari', scale=ion.vt,shift=0,symbol=r'$u_{\parallel i}/v_{ti}$',units='')
simulation.set_normalization(key='Tpare', scale=Tref_e,shift=0,symbol=r'$T_{\parallel e}$',units='eV')
simulation.set_normalization(key='Tpari', scale=Tref_i,shift=0,symbol=r'$T_{\parallel i}$',units='eV')
simulation.set_normalization(key='Tperpe',scale=Tref_e,shift=0,symbol=r'$T_{\perp e}$',units='eV')
simulation.set_normalization(key='Tperpi',scale=Tref_i,shift=0,symbol=r'$T_{\perp i}$',units='eV')

In [ ]:
#---- Compute particle loss due to fluctuations over time in the simulation
from classes import Frame

def compute_GBloss(sim,species,tf,ix=0,compute_bxgradBoB2=True):
    if compute_bxgradBoB2:
        sim.geom_param.compute_bxgradBoB2()
    # Initialize perpendicular pressure (build as a product of n and Tperp)
    pperp = 1.0
    for field in ['n', 'Tperp']:
        field_name = field + species.name[0]
        frame = Frame(sim, field_name, tf, load=True)
        pperp *= frame.values[ix, :, :]
    # Convert pressure to Joules/m3
    pperp *= sim.phys_param.eV
    # Calculate GB loss for this time frame
    GBloss_z = np.trapz(pperp * sim.geom_param.bxgradBoB2[0, ix, :, :] / species.q, x=sim.geom_param.y, axis=0)
    GBloss   = np.trapz(GBloss_z, x=sim.geom_param.z, axis=0)
    return GBloss, frame.time

def get_GBloss_t(sim, species, twindow, ix=0):
    """
    Compute the grad-B (GB) particle loss over time for a given species.
    
    Parameters:
    - sim: Simulation object (with all geometry param and units)
    - species: Species object (e.g., ion, electron)
    - twindow: List of time frames to evaluate
    - ix: Index of the flux surface (default=0)
    
    Returns:
    - GBlss_t: List of GB loss values over time
    - time: List of corresponding time points
    """
    time, GBloss_t = [], []
    # Precompute vGB_x for the given flux surface
    sim.geom_param.compute_bxgradBoB2()
    # Loop over time frames in twindow
    for tf in twindow:
        GBloss, t = compute_GBloss(sim,species,tf,ix=0,compute_bxgradBoB2=False)
        # Append corresponding GBloss value and time
        GBloss_t.append(GBloss)
        time.append(t)
    return GBloss_t, time
twindow = tfs
GBloss_i_t, time = get_GBloss_t(
    sim=simulation,
    species=ion,
    twindow=twindow,
    ix=0)

GBloss_e_t, time = get_GBloss_t(
    sim=simulation,
    species=elc,
    twindow=twindow,
    ix=0)

#---- Evaluate the particle input from the numerical gradB source
simulation.set_GBsource(    
    n_srcGB     = 3.185e23,
    T_srcGB     = 100*simulation.phys_param.eV,
    x_srcGB     = 0,
    sigma_srcGB = 10*simulation.get_rho_s(),
    bfac_srcGB  = 1.4,
    temp_model  = 'constant',
    dens_model  = 'singaus'
)
GBs = simulation.GBsource
# Evaluate the artificial gradB source
GBs_z   = [GBs.dens_source(0,z_) for z_ in simulation.geom_param.z]
GBs_tot = np.trapz(GBs_z,x=simulation.geom_param.z,axis=0)
GBsrc_t   = [GBs_tot for t_ in time]

In [ ]:
#--- Compare the source and loss along time
plt.plot(time,[-g for g in GBloss_e_t],label=r'$-S_{\nabla B e, loss}$')
plt.plot(time,[-g for g in GBloss_i_t],label=r'$-S_{\nabla B i, loss}$')
# plt.plot(time,np.abs(GBsrc_t),label=r'$|S_{\nabla B, source}|$',color='black',linestyle='--')
plt.ylabel(r'#particles')
plt.xlabel(r'$t$ ($\mu$s)')
plt.title('Particle loss at the inner flux surface')
# plt.yscale('log')
plt.legend()